# Ideal gas equation of state using grand canonical ensemble transition-matrix Monte Carlo

In this example, the ideal gas equation of state is obtained as a test of the flat histogram method.

In [1]:
import feasst as fst

monte_carlo = fst.MonteCarlo()
monte_carlo.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": "8"})),
                                  fst.args({"particle_type": fst.install_dir()+'/forcefield/data.atom'})))
monte_carlo.add(fst.Potential(fst.MakeDontVisitModel()))
criteria = fst.MakeFlatHistogram(
    fst.MakeMacrostateNumParticles(fst.Histogram(fst.args({"width": "1", "min": "0", "max": "50"}))),
    fst.MakeTransitionMatrix(fst.args({"min_sweeps": "100"})),
    fst.args({"beta": str(1./1.2), "chemical_potential": "-3"}))
monte_carlo.set(criteria)
monte_carlo.add(fst.MakeTrialTransfer(fst.args({"particle_type": "0"})))
monte_carlo.add(fst.MakeCriteriaUpdater(fst.args({"steps_per": str(1e5)})))
monte_carlo.add(fst.MakeCriteriaWriter(fst.args({"steps_per": str(1e5), "file_name": "id_fh.txt"})))

In [2]:
%%time
monte_carlo.run_until_complete()

CPU times: user 14.1 s, sys: 35.7 ms, total: 14.1 s
Wall time: 14.1 s


Check the ideal gas relationship, $\beta P = \rho$

In [8]:
import numpy as np

volume = monte_carlo.configuration().domain().volume()
gce = fst.GrandCanonicalEnsemble(criteria)
print('rho beta*P difference')
for delta_conjugate in np.arange(-5, 5, 0.1):
        lnpirw = gce.reweight(delta_conjugate)
        if lnpirw.value(lnpirw.size()-1) < -6:
            betaPV = gce.betaPV()
            N = gce.average_macrostate()
            print(N, betaPV, np.abs(1 - betaPV/N))
            assert(np.abs(1 - betaPV/N) < 1e-2)

rho beta*P difference
0.28302923815736475 0.28301519588466023 4.9614212283999315e-05
0.312795905692762 0.31278165641233363 4.555456183741313e-05
0.3456924258541493 0.34567867078436026 3.978990790731718e-05
0.3820476496873907 0.3820353934099391 3.208049430913906e-05
0.42222493578998094 0.422215560478481 2.220454242563008e-05
0.4666257761980295 0.46662111893635566 9.98072097890823e-06
0.5156938124331798 0.5156962371964694 4.701943733254055e-06
0.5699192885900839 0.5699317388859424 2.1845717644186635e-05
0.6298439954414403 0.629870005971223 4.129678137920756e-05
0.6960667676720487 0.6961104033961372 6.268899208405898e-05
0.769249605372934 0.7693152840242825 8.538015605052962e-05
0.8501245004218537 0.850216640263716 0.00010838393884249342
0.9395010575618238 0.9396234772808719 0.00013030290712578996
1.038275007582188 1.0384299920915205 0.000149271154752606
1.1474377141387466 1.14762465280196 0.00016291835357162832
1.2680867739661859 1.268300282398902 0.00016837052250640028
1.401437799657743

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!